In [7]:
import asyncio
from dataclasses import dataclass
import os
from typing import Literal

from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.runtime import Runtime

In [4]:
load_dotenv(override=True)

True

In [10]:
MODEL_PLATFORM = os.getenv("MODEL_PLATFORM")
MODEL = os.getenv("OPENAI_CHAT_MODEL")

In [5]:
@dataclass
class CustomContext:
    tools: list[Literal['weather', 'compass']]

In [8]:
@tool
async def weather() -> str:
    """Returns the current weather conditions."""
    return "It's nice and sunny"


In [9]:
@tool
async def compass() -> str:
    """Returns the direction the user is facing"""
    return "North"

In [11]:
model = init_chat_model(f"{MODEL_PLATFORM}:{MODEL}")

In [12]:
async def configure_model(state: AgentState, runtime: Runtime[CustomContext]):
    """Configure the model with tools based on the runtime context."""
    selected_tools = [tool for tool in [weather, compass] if tool.name in runtime.context.tools]
    return model.bind_tools(selected_tools)

In [22]:
agent = create_react_agent(
    # Dynamically configure the model with tools based on runtime context
    configure_model,
    # Initialize with all tools available
    tools=[weather, compass]
)

In [23]:
output = await agent.ainvoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Who are you and what tools do you have access to?"
            }
        ]
    },
    # only enable the weather tool
    context=CustomContext(tools=['weather'])
)

In [28]:
output['messages'][-1].content

'I am an AI designed to assist with a variety of tasks. I have access to a set of tools that allow me to perform specific functions. Currently, I can use the following tool:\n\n1. **Weather Tool**: This tool allows me to get the current weather conditions.\n\nAdditionally, I can execute multiple tools in parallel if needed, using a multi-tool function. However, the range of tools I can use is focused on querying specific information, such as weather data.'